In [2]:

import geopandas
import fiona
import sys
from shapely.geometry import shape, GeometryCollection, Point, MultiPolygon, Polygon, asShape
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import shapefile as shp
from shapely import wkt
import matplotlib.pyplot as plt
import seaborn as sns



c:\Users\parae\anaconda3\envs\TEST\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [23]:

column = ["ground_amplitude", "flooding_risk", "danger_level", "ground_type", "altitude"
, "rain_day", "rain_hour", "longitude", "latitude", "processo_ocorrencia", "confirmado", "solicitacao_bairro", 
"solicitacao_descricao"]
df = pd.read_csv("ground_amplitude.csv", usecols=column)


def junction(row):
    if ((row['processo_ocorrencia'] == "Imoveis com Danos/Risco"  and ("barreira" in row['solicitacao_descricao'].lower()))
     or row['processo_ocorrencia'] == "Deslizamentos de Barreiras"
    ):
        return 1 #"ImoveisDanosRiscos_Deslizamentos"

    else:
        return  0#row['processo_ocorrencia']

"""
case "Imoveis com Danos/Risco":
            return "Deslizamento_ImoveisEmRisco"
        case "Deslizamentos de Barreiras":
            return "Deslizamento_ImoveisEmRisco"
"""


'\ncase "Imoveis com Danos/Risco":\n            return "Deslizamento_ImoveisEmRisco"\n        case "Deslizamentos de Barreiras":\n            return "Deslizamento_ImoveisEmRisco"\n'

In [25]:
df['filtered_ocorrencia'] = df.apply(lambda row: junction(row), axis=1)

df['filtered_ocorrencia'].head(20)


0     0
1     0
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     1
10    0
11    0
12    0
13    1
14    0
15    1
16    0
17    0
18    1
19    0
Name: filtered_ocorrencia, dtype: int64

In [26]:
#df = df.dropna()
#df.drop_duplicates()
df.pop("solicitacao_descricao")


df.info()

#df.to_csv(r'C:\Users\parae\Documents\barreiras\barreiras\processing\slope\df.csv',
#            index=False, header=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5125 entries, 0 to 5124
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   solicitacao_bairro   5125 non-null   object 
 1   processo_ocorrencia  5125 non-null   object 
 2   confirmado           5125 non-null   int64  
 3   latitude             5125 non-null   float64
 4   longitude            5125 non-null   float64
 5   rain_hour            4855 non-null   float64
 6   rain_day             4850 non-null   float64
 7   altitude             4816 non-null   float64
 8   ground_type          5125 non-null   object 
 9   danger_level         5038 non-null   object 
 10  flooding_risk        5125 non-null   object 
 11  ground_amplitude     5038 non-null   object 
 12  filtered_ocorrencia  5125 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 520.6+ KB


In [27]:
le = LabelEncoder()

proc_ocorr = le.fit_transform(df['processo_ocorrencia'])
gro_type = le.fit_transform(df['ground_type'])
dan_level = le.fit_transform(df['danger_level'])
flo_risk = le.fit_transform(df['flooding_risk'])
gro_ampl = le.fit_transform(df['ground_amplitude'])
filt_ocorr = le.fit_transform(df['filtered_ocorrencia'])
soli_bair = le.fit_transform(df['solicitacao_bairro'])

df['processo_ocorrencia'] = proc_ocorr
df['ground_type'] = gro_type
df['danger_level'] = dan_level
df['flooding_risk'] = flo_risk
df['ground_amplitude'] = gro_ampl
df['filtered_ocorrencia'] = filt_ocorr
df['solicitacao_bairro'] = soli_bair
df.corr()

,solicitacao_bairro,processo_ocorrencia,confirmado,latitude,longitude,rain_hour,rain_day,altitude,ground_type,danger_level,flooding_risk,ground_amplitude,filtered_ocorrencia
solicitacao_bairro,1.000000,-0.044464,0.106565,0.054654,-0.357189,-0.019564,-0.002000,0.255135,-0.083123,0.124137,0.159834,0.190975,0.113224
processo_ocorrencia,-0.044464,1.000000,-0.192898,-0.087241,0.084187,-0.051726,-0.081135,-0.147684,-0.008691,-0.086200,-0.129640,-0.150545,-0.076592
confirmado,0.106565,-0.192898,1.000000,0.123456,-0.157932,0.119458,0.240398,0.295237,0.013647,0.134548,0.320437,0.338024,0.829541
latitude,0.054654,-0.087241,0.123456,1.000000,0.358119,0.062914,0.093836,0.060749,0.014970,0.012142,0.154777,0.190285,0.139153
longitude,-0.357189,0.084187,-0.157932,0.358119,1.000000,-0.000487,-0.036961,-0.389858,0.141330,-0.197454,-0.337443,-0.349438,-0.181931
rain_hour,-0.019564,-0.051726,0.119458,0.062914,-0.000487,1.000000,0.622397,0.055334,0.026156,0.028577,0.059885,0.064709,0.104376
rain_day,-0.002000,-0.081135,0.240398,0.093836,-0.036961,0.622397,1.000000,0.108010,0.016706,0.059876,0.129947,0.136426,0.224861
altitude,0.255135,-0.147684,0.295237,0.060749,-0.389858,0.055334,0.108010,1.000000,0.050678,0.442138,0.649369,0.704172,0.333944
ground_type,-0.083123,-0.008691,0.013647,0.014970,0.141330,0.026156,0.016706,0.050678,1.000000,-0.028567,-0.041102,-0.028337,-0.002333
danger_level,0.124137,-0.086200,0.134548,0.012142,-0.197454,0.028577,0.059876,0.442138,-0.028567,1.000000,0.406320,0.460066,0.157577
